In [1]:
!git clone https://github.com/nickxir12/MyCLIP_first_repo.git /kaggle/working/open-clip

Cloning into '/kaggle/working/open-clip'...
remote: Enumerating objects: 3693, done.
remote: Counting objects: 100% (3693/3693), done.
remote: Compressing objects: 100% (1422/1422), done.
remote: Total 3693 (delta 2238), reused 3624 (delta 2169), pack-reused 0 (from 0)
Receiving objects: 100% (3693/3693), 16.75 MiB | 31.42 MiB/s, done.
Resolving deltas: 100% (2238/2238), done.


In [80]:
# Step 1: Delete the old repo (if it exists)
!rm -rf /kaggle/working/open-clip  

# Step 2: Clone the latest version from GitHub
!git clone https://github.com/nickxir12/MyCLIP_first_repo.git /kaggle/working/open-clip

Cloning into '/kaggle/working/open-clip'...
remote: Enumerating objects: 3731, done.
remote: Counting objects: 100% (3731/3731), done.
remote: Compressing objects: 100% (1454/1454), done.
remote: Total 3731 (delta 2262), reused 3644 (delta 2175), pack-reused 0 (from 0)
Receiving objects: 100% (3731/3731), 16.76 MiB | 35.75 MiB/s, done.
Resolving deltas: 100% (2262/2262), done.


In [3]:
#!pip install open_clip_torch
!pip install braceexpand
!pip install webdataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 3.0 MB/s eta 0:00:00


In [4]:
!pip install -r /kaggle/working/open-clip/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00


In [5]:
import os
import sys
sys.path.append("/kaggle/working/open-clip/src")
sys.path.append("/kaggle/working/open-clip/src/open_clip")
sys.path.append("/kaggle/working/open-clip/src/open_clip_train/my_metrics")

In [6]:
# import importlib
# import my_metrics
# importlib.reload(my_metrics)
# print(dir(my_metrics))  # Now should include 'batch', 'get_all_embeddings', etc.

In [42]:
from my_metrics import compute_consistency_score,evaluate_model

#Does below work?
from my_metrics import batch,get_all_embeddings,itm_eval,compute_consistency_score

In [8]:
import open_clip
import open_clip_train

In [9]:
import numpy as np
from PIL import Image
from tqdm import tqdm
import json

In [10]:
import torch
import os
import open_clip
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [11]:
#open_clip.list_pretrained()

In [12]:
#                Preparing train datasaet

In [13]:
import json
import pandas as pd

# Load Karpathy JSON file
karpathy_json_path = "/kaggle/input/karpathy-splits/dataset_flickr30k.json"
with open(karpathy_json_path, "r") as f:
    karpathy_data = json.load(f)

# Extract training set
train_data = [item for item in karpathy_data["images"] if item["split"] == "train"]

# Prepare data for CSV
train_records = []
for item in train_data:
    img_filename = f"/kaggle/input/flickr30k/Images/{item['filename']}"
    for sentence in item["sentences"]:
        caption = sentence["raw"]
        train_records.append({"image": img_filename, "caption": caption})

# Convert to DataFrame
df_train = pd.DataFrame(train_records)

# Save to CSV (formatted properly)
csv_path = "/kaggle/working/train_data_karpathy.csv"
df_train.to_csv(csv_path, index=True, index_label="id")

print(f"✅ CSV file saved at: {csv_path}")


✅ CSV file saved at: /kaggle/working/train_data_karpathy.csv


In [14]:
df_train.head()

,image,caption
0,/kaggle/input/flickr30k/Images/1000092795.jpg,Two young guys with shaggy hair look at their ...
1,/kaggle/input/flickr30k/Images/1000092795.jpg,"Two young, White males are outside near many b..."
2,/kaggle/input/flickr30k/Images/1000092795.jpg,Two men in green shirts are standing in a yard.
3,/kaggle/input/flickr30k/Images/1000092795.jpg,A man in a blue shirt standing in a garden.
4,/kaggle/input/flickr30k/Images/1000092795.jpg,Two friends enjoy time spent together.


In [15]:
karpathy_json_path = "/kaggle/input/karpathy-splits/dataset_flickr30k.json"
with open(karpathy_json_path, "r") as f:
    karpathy_data = json.load(f)

# Extract test set
test_data = [item for item in karpathy_data["images"] if item["split"] == "test"]

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [17]:
#Standard loading
# model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
# model = model.to(device)
# model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
# tokenizer = open_clip.get_tokenizer('ViT-B-32')

In [18]:
# # FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
# model_clip_3M, _, preprocess = open_clip.create_model_and_transforms(
#     model_name="RN50",
#     pretrained=None,  # Don't load default weights
#     precision='fp32', # or 'amp' for mixed precision
#     device=device
# )

In [19]:
# # Load the checkpoint - OG CLIP - 3M
# ckpt = torch.load("/kaggle/input/clip-3m-from-cyclip/clip-3M.pt/best.pt", map_location=device)
# state_dict = ckpt["state_dict"]

# # Remove 'module.' prefix from keys
# new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# # Load into your model
# model_clip_3M.load_state_dict(new_state_dict)
# model_clip_3M.to(device)
# model_clip_3M.eval()

In [20]:
# FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
# model_Cyclip_3M, _, preprocess = open_clip.create_model_and_transforms(
#     model_name="RN50",
#     pretrained=None,  # Don't load default weights
#     precision='fp32', # or 'amp' for mixed precision
#     device=device
# )

In [21]:
# # Load the checkpoint - CY CLIP - 3M
# ckpt = torch.load("/kaggle/input/cyclip-3m/CYCLIP-3M_best.pt", map_location=device)
# state_dict = ckpt["state_dict"]

# # Remove 'module.' prefix from keys
# new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# # Load into your model
# model_Cyclip_3M.load_state_dict(new_state_dict)
# model_Cyclip_3M.to(device)
# model_Cyclip_3M.eval()

In [22]:
#Tokenizer for my models
tokenizer = open_clip.get_tokenizer('RN50')

In [24]:
#FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
model_clip_final, _, preprocess = open_clip.create_model_and_transforms(
    model_name="RN50",
    pretrained=None,  # Don't load default weights
    precision='fp32', # or 'amp' for mixed precision
    device=device
)

In [48]:
# Load the checkpoint - CLIP - Final
ckpt = torch.load("/kaggle/input/clip-final/clip_best.pt", map_location=device)
state_dict = ckpt["state_dict"]

# Remove 'module.' prefix from keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# Load into your model
model_clip_final.load_state_dict(new_state_dict)
model_clip_final.to(device)
model_clip_final.eval()

<ipython-input-48-74080bc77d25>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/kaggle/input/clip-final/clip_best.pt", map_location=device)


CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (

In [49]:
torch.save(new_state_dict, '/kaggle/working/clip_final.pt')


In [50]:
#FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
model_Cyclip_final, _, preprocess = open_clip.create_model_and_transforms(
    model_name="RN50",
    pretrained=None,  # Don't load default weights
    precision='fp32', # or 'amp' for mixed precision
    device=device
)

In [51]:
# Load the checkpoint - CY CLIP - Final
ckpt = torch.load("/kaggle/input/cyclip-final/cyclip_best.pt", map_location=device)
state_dict = ckpt["state_dict"]

# Remove 'module.' prefix from keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# Load into your model
model_Cyclip_final.load_state_dict(new_state_dict)
model_Cyclip_final.to(device)
model_Cyclip_final.eval()

<ipython-input-51-681b61487b99>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/kaggle/input/cyclip-final/cyclip_best.pt", map_location=device)


CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (

In [52]:
torch.save(new_state_dict, '/kaggle/working/Cyclip_final.pt')


In [29]:
# open_clip.list_pretrained()

In [ ]:
#                Models loaded are model_Cyclip_final and model_clip_final
#                Will further fine them below
                    # Will fine tune
                    #     --> as usual
                    #     --> with Dino regularizor

In [82]:
#          TO FINE TUNE
!python /kaggle/working/open-clip/src/open_clip_train/main.py \
    --train-data /kaggle/working/train_data_karpathy.csv \
    --name 'og_fine_tuned_Clip_1_epoch' \
    --dataset-type csv \
    --csv-img-key image \
    --csv-caption-key caption \
    --csv-separator "," \
    --model RN50  \
    --pretrained /kaggle/working/clip_final.pt \
    --batch-size 48 \
    --lr 5e-6 \
    --warmup 1000 \
    --epochs 1 \
    --lr-scheduler cosine \
    --precision amp \
    --workers 4 \
    --logs "logs" \
    --logs "checkpoints" \
    --save-frequency 1 \
    --seed 42 \

    #--lambda_dino 0.25 \
    #--use_soft_labels \
    #--soft_temprature 0.05 \
    #--alpha 0  \
    #use_dino_reg

2025-03-30 20:28:22.037817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-30 20:28:22.057658: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-30 20:28:22.063557: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using device: cuda
['/kaggle/working/open-clip/src/open_clip_train/main.py', '--train-data', '/kaggle/working/train_data_karpathy.csv', '--name', 'og_fine_tuned_Clip_1_epoch', '--dataset-type', 'csv', '--csv-img-key', 'image', '--csv-caption-key', 'caption', '--csv-separator', ',', '--model', 'RN50', '--pretrained', '/kaggle/working/clip_final.pt', '--batch-size'

In [81]:
#!rm -rf checkpoints/og_fine_tuned_Clip_1_epoch

In [59]:
!ls checkpoints/

og_fine_tuned_Clip_1_epoch


In [89]:
import torch
import open_clip

# Path to your trained checkpoint
checkpoint_path = "checkpoints/og_fine_tuned_Clip_1_epoch/checkpoints/epoch_1.pt"

# 1. Create the model architecture from scratch (without loading pretrained weights)
model_og_fine_tuned_Clip_1_epoch, preprocess_ft, preprocess_val = open_clip.create_model_and_transforms(
    "RN50", pretrained=None
)

# 2. Load the trained checkpoint manually
checkpoint = torch.load(checkpoint_path, map_location="cpu",weights_only=True)

# If the state_dict is nested under a key (like "state_dict"), adjust accordingly
if "state_dict" in checkpoint:
    checkpoint = checkpoint["state_dict"]

# Load checkpoint weights into model
model_og_fine_tuned_Clip_1_epoch.load_state_dict(checkpoint, strict=False)  # Use strict=False to avoid missing keys error

# 3. Move model to GPU (if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_og_fine_tuned_Clip_1_epoch = model_og_fine_tuned_Clip_1_epoch.to(device)
model_og_fine_tuned_Clip_1_epoch.eval()
print("Model loaded successfully!")

Model loaded successfully!


In [87]:
#          TO FINE TUNE
!python /kaggle/working/open-clip/src/open_clip_train/main.py \
    --train-data /kaggle/working/train_data_karpathy.csv \
    --name 'dino_fine_tuned_Clip_1_epoch' \
    --dataset-type csv \
    --csv-img-key image \
    --csv-caption-key caption \
    --csv-separator "," \
    --model RN50  \
    --pretrained /kaggle/working/clip_final.pt \
    --batch-size 48 \
    --lr 5e-6 \
    --warmup 1000 \
    --epochs 1 \
    --lr-scheduler cosine \
    --precision amp \
    --workers 4 \
    --logs "logs" \
    --logs "checkpoints" \
    --save-frequency 1 \
    --seed 42 \
    --lambda_dino 0.3 \
    --soft_temprature 0.05 \
    --alpha 0  \
    --use_dino_reg 

2025-03-30 20:58:34.362439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-30 20:58:34.383811: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-30 20:58:34.390057: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using device: cuda
['/kaggle/working/open-clip/src/open_clip_train/main.py', '--train-data', '/kaggle/working/train_data_karpathy.csv', '--name', 'dino_fine_tuned_Clip_1_epoch', '--dataset-type', 'csv', '--csv-img-key', 'image', '--csv-caption-key', 'caption', '--csv-separator', ',', '--model', 'RN50', '--pretrained', '/kaggle/working/clip_final.pt', '--batch-siz

In [86]:
#!rm -rf checkpoints/dino_fine_tuned_Clip_1_epoch

In [90]:
# Path to your trained checkpoint
checkpoint_path = "checkpoints/dino_fine_tuned_Clip_1_epoch/checkpoints/epoch_1.pt"

# 1. Create the model architecture from scratch (without loading pretrained weights)
model_dino_fine_tuned_Clip_1_epoch, preprocess_ft, preprocess_val = open_clip.create_model_and_transforms(
    "RN50", pretrained=None
)

# 2. Load the trained checkpoint manually
checkpoint = torch.load(checkpoint_path, map_location="cpu",weights_only=True)

# If the state_dict is nested under a key (like "state_dict"), adjust accordingly
if "state_dict" in checkpoint:
    checkpoint = checkpoint["state_dict"]

# Load checkpoint weights into model
model_dino_fine_tuned_Clip_1_epoch.load_state_dict(checkpoint, strict=False)  # Use strict=False to avoid missing keys error

# 3. Move model to GPU (if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_dino_fine_tuned_Clip_1_epoch = model_dino_fine_tuned_Clip_1_epoch.to(device)
model_dino_fine_tuned_Clip_1_epoch.eval()
print("Model loaded successfully!")

Model loaded successfully!


In [88]:
import shutil

shutil.make_archive("/kaggle/working/cyclip_checkpoints", 'zip', "/kaggle/working/checkpoints")


'/kaggle/working/cyclip_checkpoints.zip'

In [ ]:
#      EVALUATING BELOW

In [36]:
test_data = [item for item in karpathy_data["images"] if item["split"] == "test"]
all_captions = []
all_images = []
for item in test_data:
    for sentence in item["sentences"]:
        all_captions.append(sentence["raw"])
        all_images.append(item["filename"])

In [38]:
# Compute embeddings
#FROM CYCLIP

image_root = "/kaggle/input/flickr30k/Images"
text_embeds, image_embeds = get_all_embeddings(
    model_Cyclip_final, all_captions, all_images, root=image_root,
    preprocess=preprocess, tokenizer=tokenizer, device=device , batch_size=48
)

Encoding batches: 100%|██████████| 105/105 [01:17<00:00,  1.35it/s]


In [43]:
#FROM CYCLIP
# Evaluate
results = itm_eval(text_embeds, image_embeds)
print("📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.2f}")

📊 Evaluation Results:
txt_r1: 88.24
txt_r5: 94.00
txt_r10: 96.00
txt_r_mean: 92.75
img_r1: 30.42
img_r5: 57.10
img_r10: 68.82
img_r_mean: 52.11
r_mean: 72.43


In [44]:
image_root = "/kaggle/input/flickr30k/Images"
text_embeds, image_embeds = get_all_embeddings(
    model_clip_final, all_captions, all_images, root=image_root,
    preprocess=preprocess, tokenizer=tokenizer, device=device , batch_size=48
)

Encoding batches: 100%|██████████| 105/105 [01:08<00:00,  1.54it/s]


In [45]:
#FROM CYCLIP
# Evaluate
results = itm_eval(text_embeds, image_embeds)
print("📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.2f}")

📊 Evaluation Results:
txt_r1: 88.10
txt_r5: 93.84
txt_r10: 95.90
txt_r_mean: 92.61
img_r1: 30.54
img_r5: 56.92
img_r10: 68.26
img_r_mean: 51.91
r_mean: 72.26


In [ ]:
#FOR FINE TUNED CLIP

In [91]:
# Compute embeddings

image_root = "/kaggle/input/flickr30k/Images"
text_embeds, image_embeds = get_all_embeddings(
    model_og_fine_tuned_Clip_1_epoch, all_captions, all_images, root=image_root,
    preprocess=preprocess, tokenizer=tokenizer, device=device , batch_size=48
)

Encoding batches: 100%|██████████| 105/105 [01:07<00:00,  1.57it/s]


In [92]:
# Evaluate
results = itm_eval(text_embeds, image_embeds)
print("📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.2f}")

📊 Evaluation Results:
txt_r1: 90.02
txt_r5: 95.62
txt_r10: 97.26
txt_r_mean: 94.30
img_r1: 36.06
img_r5: 65.32
img_r10: 75.84
img_r_mean: 59.07
r_mean: 76.69


In [94]:
image_root = "/kaggle/input/flickr30k/Images"
text_embeds, image_embeds = get_all_embeddings(
    model_dino_fine_tuned_Clip_1_epoch, all_captions, all_images, root=image_root,
    preprocess=preprocess, tokenizer=tokenizer, device=device , batch_size=48
)

Encoding batches: 100%|██████████| 105/105 [01:07<00:00,  1.55it/s]


In [95]:
# Evaluate
results = itm_eval(text_embeds, image_embeds)
print("📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.2f}")

📊 Evaluation Results:
txt_r1: 89.98
txt_r5: 95.54
txt_r10: 97.22
txt_r_mean: 94.25
img_r1: 36.34
img_r5: 65.74
img_r10: 76.02
img_r_mean: 59.37
r_mean: 76.81


In [ ]:
# CONSISTENCY METRICS BELOW

In [105]:
test_records = []
for item in test_data:
    img_filename = f"/kaggle/input/flickr30k/Images/{item['filename']}"
    for sentence in item["sentences"]:
        caption = sentence["raw"]
        test_records.append({"image": img_filename, "caption": caption})

In [109]:
from torch.utils.data import Dataset

class CyCLIPDataset(Dataset):
    def __init__(self, records, image_transform):
        self.records = records
        self.image_transform = image_transform

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        record = self.records[idx]
        image = self.image_transform(Image.open(record["image"]).convert("RGB"))
        text = tokenizer(record["caption"]).squeeze(0)  # remove batch dim
        return image, text

In [111]:
from torch.utils.data import DataLoader

test_dataset = CyCLIPDataset(test_records, preprocess)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [112]:
# compute_consistency_score calculate mean of similarities

# CLIP final with og Fine tuning

score = compute_consistency_score(model_dino_fine_tuned_Clip_1_epoch, test_loader, device)

print(f"Consistency Score: {score:.4f}")

Consistency Score: 0.4355


In [113]:
# CLIP final with og Fine tuning

score = compute_consistency_score(model_og_fine_tuned_Clip_1_epoch, test_loader, device)

print(f"Consistency Score: {score:.4f}")

Consistency Score: 0.4334


In [117]:
# CLIP final

score = compute_consistency_score(model_clip_final, test_loader, device)

print(f"Consistency Score: {score:.4f}")

Consistency Score: 0.0070


In [116]:
# CYCLIP final 

score = compute_consistency_score(model_Cyclip_final, test_loader, device)

print(f"Consistency Score: {score:.4f}")

Consistency Score: -0.0388
